In [1]:
import numpy as np
import pickle
import dimod
import dwave_token
import dwave.inspector

from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
from dwave.system import DWaveCliqueSampler
from dwave.system import LeapHybridCQMSampler

# automatically generated embedding:
sampler = EmbeddingComposite(DWaveSampler(token=dwave_token.value))
clique_sampler = DWaveCliqueSampler(token=dwave_token.value)
hybrid_sampler = LeapHybridCQMSampler(token=dwave_token.value)

In [41]:
def make_QUBO_upper_triangle(Q):
    Q = np.array(Q)
    return(np.transpose(Q - np.triu(Q)) + np.triu(Q))

def weighted_clustering_heuristic(distances, k, delta, n_max, solver='DWAVE', time_limit=0, penalty=False, fix_variables=False):
    N = len(distances)
    if penalty == False and (solver != 'hybrid' and solver != 'CPLEX'):
        penalty = True
    if penalty == True:
        penalty = np.max(distances)* (N - k)
    if solver == 'CPLEX':
        model = pyo.AbstractModel()
        model.points = pyo.RangeSet(0, N-1)
        model.unique_pairs = pyo.Set(initialize=model.points*model.points, filter = lambda model, p1, p2: p1 < p2)
        model.clusters = pyo.RangeSet(1,k)
        model.d = pyo.Param(model.unique_pairs, initialize=lambda model, p1, p2: distances[p1,p2])
        model.lambda_n = pyo.Param(default=0, within=pyo.NonNegativeReals)
        model.x = pyo.Var(model.points*model.clusters, within=pyo.Binary)
        def obj_0(model,g):
            return(sum(model.d[i,j]*model.x[i,g]*model.x[j,g] for i,j in model.unique_pairs))
        def N_cluster(model,g):
            N_g = sum(model.x[i,g] for i in model.points)
            return(N_g * (N_g-1))
        def uniqueness(model,i):
            return(sum(model.x[i,g] for g in model.clusters) == 1)
        if penalty:
            model.obj = pyo.Objective(rule=lambda model: sum(obj_0(model,g) - model.lambda_n * N_cluster(model,g) for g in model.clusters) + penalty * sum((sum(model.x[i,g] for g in model.clusters) - 1)**2 for i in model.points))
        else:
            model.obj = pyo.Objective(rule=lambda model: sum(obj_0(model,g) - model.lambda_n * N_cluster(model,g) for g in model.clusters))
            model.uniqueness = pyo.Constraint(model.points, rule=uniqueness)
        opt = pyo.SolverFactory('cplex', tee=False)
        if time_limit != 0:
            opt.options['timelimit'] = time_limit
    else:
        QUBO_core = np.triu([[distances[i,j] if g == l else 0 for g in range(k) for i in range(N)] for l in range(k) for j in range(N)])
        bqm_core = dimod.BinaryQuadraticModel.from_qubo(QUBO_core)
        if solver != 'hybrid':
            QUBO_penalty = penalty * make_QUBO_upper_triangle([[(-1 if g == l else 1) if i == j else 0 for g in range(k) for i in range(N)] for l in range(k) for j in range(N)])
            bqm_core.add_linear_from_array(np.diag(QUBO_penalty))
            bqm_core.add_quadratic_from_dense(QUBO_penalty - np.diag(np.diag(QUBO_penalty)))
            bqm_core.offset += penalty*N
        def obj_0(values,g):
            return(sum(distances[i,j] * values[i,g] * values[j,g] for i in range(N) for j in range(i,N)))
        def N_cluster(values,g):
            N_g = sum(values[i,g] for i in range(N))
            return(N_g * (N_g-1))
        if solver == 'clique':
            DWavesampler = clique_sampler
        elif solver == 'hybrid':
            def uniqueness(i):
                return([g*N + i for g in range(k)])
        else:
            DWavesampler = sampler
    lambdas = [0]
    ## start loop:
    for n in range(n_max+1):
        print('n = ', n)
        lambda_n = lambdas[-1]
        if solver == 'CPLEX':
            instance = model.create_instance(data={None: {'lambda_n': {None: lambda_n}}})
            if fix_variables:
                for g in model.clusters:
                    instance.x[0,g].fix(int(g==1))
            results = opt.solve(instance)
            lambda_n1 = sum(pyo.value(obj_0(instance,g)/N_cluster(instance,g)) if pyo.value(sum(instance.x[i,g] for i in instance.points)) > 1 else 0 for g in instance.clusters)
        else:
            QUBO_lambda = - lambda_n * make_QUBO_upper_triangle([[1 if (g == l and i != j) else 0 for g in range(k) for i in range(N)] for l in range(k) for j in range(N)])
            bqm = dimod.as_bqm(bqm_core, copy=True)
            bqm.add_linear_from_array(np.diag(QUBO_lambda))
            bqm.add_quadratic_from_dense(QUBO_lambda - np.diag(np.diag(QUBO_lambda)))
            if fix_variables:
                bqm.fix_variables({g*N:int(g==0) for g in range(k)})
            if solver == 'hybrid':
                cqm = dimod.ConstrainedQuadraticModel.from_bqm(bqm)
                for i in range(int(fix_variables), N):
                    cqm.add_discrete(uniqueness(i), label='uniqueness' + str(i))
                sampleset = hybrid_sampler.sample_cqm(
                    cqm,
                    time_limit = time_limit,
                )
            else:
                chain_strength = max(bqm.quadratic.values())
                sampleset = DWavesampler.sample(
                    bqm,
                    num_reads=1000,
                    chain_strength=chain_strength
                )
            if fix_variables:
                values = {(i,g): int(g==0) if i == 0 else sampleset.lowest().samples()[0][g*N+i] for g in range(k) for i in range(0,N)}
            else:
                values = {(i,g): sampleset.lowest().samples()[0][g*N+i] for g in range(k) for i in range(N)}
            lambda_n1 = sum(obj_0(values,g)/N_cluster(values,g) if sum(values[i,g] for i in range(N)) > 1 else 0 for g in range(k))
        lambdas += [lambda_n1]
        print('lambda = ',lambda_n1)
        if abs(lambdas[-1] - lambdas[-2]) <= delta:
            break
    return(lambdas)

In [42]:
distances = np.array([[0,1,1,1,np.sqrt(5)],[1,0,0,0,2],[1,0,0,0,2],[1,0,0,0,2],[np.sqrt(5),2,2,2,0]])
lambdas = weighted_clustering_heuristic(distances, k=2, delta=0, n_max=5, solver='hybrid', time_limit=5, penalty=False, fix_variables=True)
print('lambdas:', lambdas)

n =  0
lambda =  1.118033988749895
n =  1
lambda =  0.5618033988749895
n =  2
lambda =  0.25
n =  3
lambda =  0.25


ValueError: too many values to unpack (expected 2)